In [126]:
import os
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd
import random


os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())


Current Working Directory  D:\02_batch_2


### Arabic Annotation collection

In [89]:
df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)

li = []

try:

    for i,dname in enumerate(df_dirname['Folder']):
        path = r'D:/02_batch_2/'
        files = glob.glob(path + dname + '/*.txt')
        print(f'Successfully created dataframe for {dname}')
        for i,f in enumerate(files):

            parse_error = False
            file_not_found = False

            stock = os.path.basename(f)            
            temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
            temp_df['filePath'] = path + dname + '/'
            temp_df['textFileName'] = stock     
            temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
            temp_df['duration'] = temp_df.stop-temp_df.start
            li.append(temp_df)
            print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

except pd.errors.ParserError:
    parse_error = True
    temp_df = pd.read_csv(f, delimiter="\t",skiprows=4)
    li.append(temp_df)
    print(f'Error for {dname} {stock} with shape {temp_df.shape}')

else:
    li.append(temp_df)


df = pd.concat(li, axis=0)
print(df.shape)
df.head(3)

In [175]:
li = []
path = r'D:/02_batch_2/'

for i,dname in enumerate(df_dirname['Folder']):

    files = glob.glob(path + dname + '/*.txt')
    #print(f'Successfully created dataframe for {dname}')
    for i,f in enumerate(files):

        parse_error = False
        file_not_found = False

        stock = os.path.basename(f)            
        temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
        temp_df['filePath'] = path + dname + '/'
        temp_df['textFileName'] = stock     
        temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
        temp_df['duration'] = temp_df.stop-temp_df.start
        li.append(temp_df)
      #  print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

df = pd.concat(li, axis=0)
print(df.shape)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


(507921, 9)


In [177]:
df.head(2)

,speaker,gender,start,stop,transcript,filePath,textFileName,audioFileName,duration
0,speakeA,M,8.998978,23.522329,الحقوني الحقوني بطني بطني بموت ما أقدر الحقون...,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,14.523351
1,speakeA,M,23.522329,25.772074,أضحك عليكو بس أختبركو.,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,2.249745


In [ ]:
df.duration.sum()/3600

1161.3279405906094

### Video Clipping

In [184]:
hours = 180000;
duration_sum = 0;
x=0;
dest_filepath = r'D:/nf/audio/'

df_selection = pd.DataFrame(columns=['row_no','src_filepath','src_filepath_txt','src_filepath_wav','dest_filepath','dest_filepath_txt','dest_filepath_wav','duration'])

while (hours>=duration_sum):
    n = random.randint(0,len(df))
    fname = str(df.iloc[n,5] + df.iloc[n,7])
    duration_sum = duration_sum + df.iloc[n,8].astype('float32')

    df_selection[x,0] = n
    df_selection[x,1] = str(df.iloc[n,5])
    df_selection[x,2] = str(df.iloc[n,6])
    df_selection[x,3] = str(df.iloc[n,7])
    df_selection[x,4] = dest_filepath
    df_selection[x,5] = str(x) + "_" + str(df.iloc[n,6])
    df_selection[x,6] = str(x) + "_" + str(df.iloc[n,7])
    df_selection[x,7] = df.iloc[n,8].astype('float32')

    x=x+1
    print(sum)
    #my_clip = mp.AudioFileClip(fname)


df_selection.head(4) 

0.2999528646469116
1.099762499332428
10.599145472049713
22.898875772953033
23.598816514015198
25.04851007461548
45.048325061798096
72.49712610244751
80.57128381729126
81.29624307155609
83.07096314430237
86.44525718688965
93.01989936828613
94.51968288421631
98.29450130462646
98.54445573687553
99.66945061087608
109.09375938773155
111.49340423941612
137.1182287633419
155.21695885062218
158.5668521821499
167.4408298432827
170.9407852590084
172.6905160844326
198.16482219099998
213.46094664931297
220.28591975569725
235.1076532304287
236.63262602686882
239.7323393523693
241.4071851670742
250.10666236281395
251.98144707083702
281.87289986014366
282.7228032052517
283.4227116405964
287.7226121723652
289.2726017534733
307.77107778191566
311.070431381464
322.9696718752384
329.4692102968693
351.61887803673744
360.1422678530216
362.71696677803993
371.4168683588505
399.5414754450321
426.7879811823368
440.5121356546879
449.8865558207035
451.1112999022007
451.3612906187773
456.1358763426542
482.0743910

In [ ]:
fp = df.iloc[1,5]
fn = df.iloc[1,7]
fname = str(fp+fn)

my_clip = mp.AudioFileClip(fname)
clip = my_clip.subclip(df.iloc[1,2],df.iloc[1,3])

clip.write_audiofile(r"D:\nf\audio\1_"+str(fn)+".mp3")

#### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")